In [4]:
# 환경 설정

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
# 데이터 로딩

import numpy as np
import pandas as pd

X_train = pd.read_csv('gdrive/MyDrive/UCI-HAR-Dataset/train/X_train.txt', delim_whitespace=True).to_numpy()
y_train = pd.read_csv('gdrive/MyDrive/UCI-HAR-Dataset/train/y_train.txt', delim_whitespace=True).to_numpy().ravel()
X_test = pd.read_csv('gdrive/MyDrive/UCI-HAR-Dataset/test/X_test.txt', delim_whitespace=True).to_numpy()
y_test = pd.read_csv('gdrive/MyDrive/UCI-HAR-Dataset/test/y_test.txt', delim_whitespace=True).to_numpy().ravel()

In [6]:
# X_train, y_train 모양 확인
print(X_train.shape)
print(y_train.shape)

(7351, 561)
(7351,)


In [7]:
# 특성 수를 줄이기 위한 feature_selection

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

select = SelectKBest(mutual_info_classif, k=500) # 1, 10, 50, 100, 200, 300, 400, 500 을 시도해 보았으나 모든 특성을 사용할 때 가장 높은 정확도를 보임
X_train = select.fit_transform(X_train, y_train)
X_test = select.transform(X_test)

In [8]:
# # 로지스틱회귀

# from sklearn.linear_model import LogisticRegression 

# logreg = LogisticRegression()
# logreg.fit(X_train, y_train)
# print("테스트 세트 점수: {:.2f}".format(logreg.score(X_test, y_test) * len(X_test)))

In [9]:
# # SVM

# from sklearn.svm import SVC

# svc = SVC()
# svc.fit(X_train, y_train)
# print("테스트 세트 점수: {:.2f}".format(svc.score(X_test, y_test) * len(X_test)))

In [10]:
# # 랜덤포레스트

# from sklearn.ensemble import RandomForestClassifier

# rf = RandomForestClassifier()
# rf.fit(X_train, y_train)
# print("테스트 세트 점수: {:.2f}".format(rf.score(X_test, y_test) * len(X_test)))

In [11]:
# # KNN, 그리드 서치에는 값이 너무 상대적으로 낮게 나와 사용하지 않음

# from sklearn.neighbors import KNeighborsClassifier

# knn = KNeighborsClassifier()
# knn.fit(X_train, y_train)
# print("테스트 세트 점수: {:.2f}".format(knn.score(X_test, y_test) * len(X_test)))

In [14]:
# 교차검증 + 그리드 서치 0.01 ~ 10000000 까지의 파라미터 시험 결과 후 수렴하는 파라미터로 값을 줄여가며 반복함

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

pipe = Pipeline([("estimator", LogisticRegression())])

kfold = KFold(n_splits=5, shuffle=True, random_state=1)
param_grid = [{
                  'estimator' : [LogisticRegression()],
                  'estimator__C' : [13, 14, 15, 16, 17]
                  # , 'estimator__penalty' : ['l1', 'l2', 'elasticnet']
              }
              # ,
              # {
              #     'estimator' : [SVC()],
              #     'estimator__gamma': [9, 10, 11], 
              #     'estimator__C': [0.01, 0.1, 0.2]
              # }
              # {
              #     'estimator': [RandomForestClassifier()],
              #     'estimator__max_depth': [4, 6],
              #     'estimator__n_estimators': [50, 100],
              #     'estimator__min_samples_split': [50, 100]
              # }
              ]
grid = GridSearchCV(pipe, param_grid=param_grid, scoring='f1_macro', cv=kfold, n_jobs=-1)
grid.fit(X_train, y_train)

# Play an audio beep. Any audio URL will do.
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [15]:
print(grid.best_params_)
grid.best_score_

{'estimator': LogisticRegression(C=15, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False), 'estimator__C': 15}


0.9841361267451202

In [23]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import plot_confusion_matrix

mat = confusion_matrix(y_test, grid.best_estimator_.predict(X_test))
# plot_confusion_matrix(grid.best_estimator_, X_test, y_test)

In [ ]:
classification_report(y_test, grid.best_estimator_.predict(X_test))

In [18]:
              precision    recall  f1-score   support

           1       0.95      0.98      0.97       496
           2       0.96      0.94      0.95       471
           3       0.98      0.96      0.97       420
           4       0.97      0.88      0.92       491
           5       0.89      0.98      0.93       531
           6       1.00      1.00      1.00       537

    accuracy                           0.96      2946
   macro avg       0.96      0.96      0.96      2946
weighted avg       0.96      0.96      0.96      2946


SyntaxError: ignored